In [52]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [53]:
# Inicializa o Driver e abre o navegador
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\teixe\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


In [54]:
# Coleta os dados de cada loja
loja = ['Submarino',
       'Americanas',
       'Casas Bahia',
       'Magazine Luiza',
       'Amazon']
urls = ['https://www.reclameaqui.com.br/empresa/submarino/',
        'https://www.reclameaqui.com.br/empresa/americanas-com-loja-online/',
       'https://www.reclameaqui.com.br/empresa/casas-bahia-loja-online/',
       'https://www.reclameaqui.com.br/empresa/magazine-luiza-loja-online/',
       'https://www.reclameaqui.com.br/empresa/amazon/']
itens = []
for index, url in enumerate(urls):
    driver.get(url)
    dados = driver.find_elements_by_class_name('sc-9xbj9-0')
    reclamacoes = dados[0].text.split('%')[0]
    negociar = dados[1].text.split('%')[0]
    ind_solucao = dados[2].text.split('%')[0]
    nota_consumidor = dados[3].text
    n_reclamacoes = driver.find_elements_by_class_name('short-ammount')[0].text.split('\n')[1]
    item = {'Loja': loja[index],
        'pct_reclamacoes': reclamacoes,
        "voltariam_negociar": negociar,
        'indice_solucao': ind_solucao,
        'nota_consumidor': nota_consumidor,
        'numero_reclamacoes': n_reclamacoes,
        }
    itens.append(item)

In [60]:
# Coleta o tempo de reposta (em dias), que está em uma url diferente
itens_time = []
for index, url in enumerate(urls):
    url_time = url+'lista-reclamacoes/?pagina=1'
    driver.get(url_time)
    time_response = driver.find_elements_by_class_name('company-index-value')[3].text
    if ('dias' in time_response)&('horas' in time_response):
        el = time_response.split(' ')[0::3]
        horas = int(el[1])/24
        time_response = round(float(el[0])+horas, 2)
    else:
        time_response =  round(int(time_response.split(' ')[0])/24, 2)
    dictio = {'Tempo_Resposta':time_response}
    itens_time.append(dictio)

In [63]:
import pandas as pd
data = pd.DataFrame(itens)
data_time = pd.DataFrame(itens_time)
df = pd.concat([data, data_time], axis=1)
df

,Loja,pct_reclamacoes,voltariam_negociar,indice_solucao,nota_consumidor,numero_reclamacoes,Tempo_Resposta
0,Submarino,97.6,73,82.8,7.04,5658,3.33
1,Americanas,98.5,84.7,92.9,8.36,21120,2.33
2,Casas Bahia,99.1,63.7,85.2,6.37,26801,8.38
3,Magazine Luiza,96.8,80.3,92.7,7.76,57316,7.12
4,Amazon,99.7,81,85.1,7.84,35222,0.79


In [64]:
df.to_csv('reclame_aqui.csv', sep=';', encoding='utf-8', index=False)